# Delta Lake Time Machine & Optimization Lab

Delta Lake allows you to read, write and query data in data lakes in an efficient manner.

## In this lesson you:
* Compare different versions of a Delta table using Time Machine
* Optimize your Delta Lake to increase speed and reduce number of files

## Audience
* Primary Audience: Data Engineers
* Secondary Audience: Data Analysts and Data Scientists

## Prerequisites
* Web browser: current versions of Google Chrome, Firefox, Safari, Microsoft Edge and
Internet Explorer 11 on Windows 7, 8, or 10 (see <a href="https://docs.databricks.com/user-guide/supported-browsers.html#supported-browsers#" target="_blank">Supported Web Browsers</a>)
* Databricks Runtime 4.2 or greater
## Datasets Used
We will use online retail datasets from `/mnt/training/online_retail`

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtrainwesteurope.blob.core.windows.net/

Created user-specific database

Using the database saifahmed_k_outlook_com_db .

All done!

Because we'll be calculating some aggregates in this notebook, we'll change our partitions after shuffle from the default `200` to `8` (which is a good number for the 8 node cluster we're currently working on).

In [0]:
%python

sqlContext.setConf("spark.sql.shuffle.partitions", "8")

### Check for our previous Delta Lake tables

This lab relies upon some tables created in previous Delta Lake lessons and labs. 

If you get an error from either of the next two SQL queries, running the solution code for the "Delta-Lake-Lab-1" will build all necessary tables.

In [0]:
%sql

SELECT COUNT(*) FROM customer_counts;
SELECT COUNT(*) FROM customer_data_delta;

count(1)
65499


-sandbox
<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> **The following cell will take several minutes to execute, and is only necessary to run if you got an error in the previous cell.**

In [0]:
%run "./Includes/Delta-Lab-2-Prep"

For convenience later in this lab, the paths to the files defining our existing Delta tables are provided. You can use these paths to load the data into DataFrames, if desired, though this entire lab can be completed using SQL on the existant tables.

In [0]:
DeltaPath = userhome + "/delta/customer-data/"
CustomerCountsPath = userhome + "/delta/customer_counts/"

**Note: This lab depends upon the complete exectuion of the notebook titled "Open-Source-Delta-Lake" and the "Delta-Lake-Basics" lab. If these tables don't exist, go back and run all cells in these notebook.**

### Time Travel
Because Delta Lake is version controlled, you have the option to query past versions of the data. Let's look at the history of our current Delta table.

In [0]:
%sql
DESCRIBE HISTORY customer_data_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2024-10-20T11:57:41.000+0000,7104407279782773,saifahmed.k@outlook.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [""Country""])",null,List(2599175015539894),1019-080424-ac1xqx2j,null,WriteSerializable,false,"Map(numFiles -> 37, numOutputRows -> 65499, numOutputBytes -> 633565)",null,Databricks-Runtime/11.3.x-photon-scala2.12


Querying an older version is as easy as adding `VERSION AS OF desired_version`. Let's verify that our table from one version back still exists.

In [0]:
%sql
SELECT COUNT(*)
FROM customer_data_delta
VERSION AS OF 0

count(1)
65499


Using a single file storage system, you now have access to every version of your historical data, ensuring that your data analysts will be able to replicate their reports (and compare aggregate changes over time) and your data scientists will be able to replicate their experiments.

### Check difference between versions

You want to compare how many orders from Sweden were added by your recent UPSERT to your BI table.

Let's start by getting the total sum of our `total_orders` column where our country is Sweden.

In [0]:
# TODO
count = spark.sql(f"SELECT COUNT(*) FROM delta.`{DeltaPath}`").collect()[0][0]

Again, we can look at the history of our Delta table here.

In [0]:
%sql
DESCRIBE HISTORY customer_counts

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2024-10-20T12:17:02.000+0000,7104407279782773,saifahmed.k@outlook.com,MERGE,"Map(predicate -> [""((Country#2287 = Country#2212) AND (CustomerID#2286 = CustomerID#2211))""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(930942090424827),1019-080424-ac1xqx2j,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 4, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, executionTimeMs -> 8440, materializeSourceTimeMs -> 893, numTargetRowsInserted -> 4, scanTimeMs -> 5959, numTargetRowsUpdated -> 1205, numOutputRows -> 1213, numTargetChangeFilesAdded -> 0, numSourceRows -> 1209, numTargetFilesRemoved -> 1, rewriteTimeMs -> 1160)",null,Databricks-Runtime/11.3.x-photon-scala2.12
0,2024-10-20T12:05:38.000+0000,7104407279782773,saifahmed.k@outlook.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(930942090424827),1019-080424-ac1xqx2j,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1209, numOutputBytes -> 8070)",null,Databricks-Runtime/11.3.x-photon-scala2.12


Our original table will be version `0`. Let's write a SQL query to see how many orders we originally had from Sweden.

In [0]:
%sql
SELECT SUM(total_orders)
FROM customer_counts
VERSION AS OF 0
WHERE Country='Sweden'

sum(total_orders)
41


We can combine these two queries and get our difference, which represents our new entries.

In [0]:
%sql
SELECT SUM(total_orders) - (
  SELECT SUM(total_orders)
  FROM customer_counts
  VERSION AS OF 0
  WHERE Country='Sweden') AS new_entries
FROM customer_counts
WHERE Country='Sweden'

new_entries
41


### OPTIMIZE and ZORDER

Let's apply some of these optimizations to `../delta/customer-data/`.

Our data is partitioned by `Country`.

We want to query the data for `StockCode` equal to `22301`.

We expect this query to be slow because we have to examine ALL OF `../delta/customer-data/` to find the desired `StockCode` and not just in one or two partitions.

First, let's time the above query: you will need to form a DataFrame to pass to `preZorderQuery`.

In [0]:
# TODO
%timeit preZorderQuery = spark.sql(f"SELECT * FROM delta.`{DeltaPath}`").collect()

938 ms ± 76.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Compact the files and re-order by `StockCode`.

In [0]:
%sql
-- TODO
OPTIMIZE customer_counts
ZORDER by (CustomerID)

path,metrics
dbfs:/user/saifahmed.k@outlook.com/delta/customer_counts,"List(1, 2, List(8314, 8314, 8314.0, 1, 8314), List(1009, 8314, 4661.5, 2, 9323), 0, List(minCubeSize(107374182400), List(0, 0), List(2, 9323), 0, List(2, 9323), 1, null), 1, 2, 0, false, 0, 0, 1729453315810, 1729453318762, 4, 1, null)"


Let's time the above query again: you will need to form a DataFrame to pass to `postZorderQuery`.

In [0]:
# TODO
%timeit postZorderQuery = spark.sql(f"SELECT * FROM delta.`{DeltaPath}`").collect()

844 ms ± 51.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### OPTIMIZE your BI table

Here we'll optimize our `customer_counts` table so that we can quickly query on our `CustomerID` column.

In [0]:
%sql
OPTIMIZE customer_counts
ZORDER by (CustomerID)

path,metrics
dbfs:/user/saifahmed.k@outlook.com/delta/customer_counts,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(1, 8314), List(0, 0), 1, List(0, 0), 0, null), 0, 1, 1, false, 0, 0, 1729453394124, 1729453394600, 4, 0, null)"


Now we can easily look at which of our customers have made the most orders.

In [0]:
%sql
SELECT CustomerID, SUM(total_orders) AS total
FROM customer_counts
GROUP BY CustomerID
ORDER BY total DESC

CustomerID,total
null,50562
12748,1390
17841,962
14606,842
15311,836
14911,754
17850,594
13089,522
13081,522
18118,500


Or we can see examine those customers that operate in the most countries.

In [0]:
%sql
SELECT CustomerID, COUNT(Country) AS num_countries
FROM customer_counts
GROUP BY CustomerID
SORT BY num_countries DESC

CustomerID,num_countries
null,8
12370,2
14496,1
13090,1
14135,1
15260,1
17802,1
16995,1
13370,1
13269,1


And then look at how many orders a customer made in each of these countries.

In [0]:
%sql
SELECT Country, total_orders
FROM customer_counts
WHERE CustomerID = 20059

Country,total_orders


### Using VACUUM to clean up small files

After we run OPTIMIZE, we have a number of uncompacted files that are no longer necessary. Running VACUUM will remove these files for us.

Let's go ahead and VACUUM our `customer_data_delta` table, which points at the files in our `DeltaPath` variable.

Count number of files before `VACUUM` for `Country=Sweden`.

In [0]:
# TODO
preNumFiles = len(dbutils.fs.ls(DeltaPath))
display(preNumFiles)

25

If you try to perform an immediate `VACUUM` (using `RETAIN 0 HOURS` to clean up recently optimized files), you will get an error.

In [0]:
%sql
-- TODO
VACUUM customer_data_delta RETAIN 0 HOURS;

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-1914720889771087> in <cell line: 1>()
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-1914720889771087> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("LS0gVE9ETwpWQUNVVU0gY3VzdG9tZXJfZGF0YV9kZWx0YSBSRVRBSU4gMCBIT1VSUw==").decode())
      5     display(df)
      6     return df

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_coun

This is a helfpul error. Remember that `VACUUM` is intended for occasional garbage collection. Here we'll just demonstrating that we _can_ use it to clean up files, so we'll set our configuration to allow this operation.

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

Now we won't get an error when we run `VACUUM`.

In [0]:
%sql
VACUUM customer_data_delta RETAIN 0 HOURS;

path
dbfs:/user/saifahmed.k@outlook.com/delta/customer-data


Count how many files there are for `Country=Sweden`.

In [0]:
# TODO
postNumFiles = len(dbutils.fs.ls(DeltaPath))
display(postNumFiles)

25

Comparing our `preNumFiles` to `postNumFiles`, we can see that this number has reduced.

This notebook is focused on optimizing and managing data in a **Delta Lake**, which is an open-source storage layer that helps with efficient data processing and version control in data lakes.

Here’s a breakdown of the key concepts covered:

### 1. **Delta Lake**
   - **Delta Lake** is a tool that allows you to store large amounts of data in a way that supports version control and transactional reliability (ACID compliance). It enables efficient **reading**, **writing**, and **querying** of data stored in data lakes.

### 2. **Time Travel**
   - **Time Travel** in Delta Lake allows you to look at different **versions** of your data at specific points in time. This means you can query data as it existed at any version by using the **`VERSION AS OF`** feature.
   - Example: You query an older version of a table to see how the data looked in the past or compare changes over time.

### 3. **Optimization (OPTIMIZE & ZORDER)**
   - **Optimize**: In a Delta Lake, data is stored in multiple small files. Over time, as data is updated, the number of these small files can grow, which can slow down query performance. 
     - **Optimization** compacts these small files into larger ones, making the system faster and more efficient to query.
   - **ZORDER**: This is a method of sorting data to further speed up specific types of queries.
     - For example, if you often query data based on a specific column (e.g., **CustomerID**), you can reorder the data using **ZORDER** to make the searches more efficient by reducing the number of files that need to be checked.

### 4. **VACUUM**
   - **Vacuum** is a process that cleans up **old and unnecessary files** in Delta Lake.
   - After optimizing or making changes, there can be leftover data files that are no longer needed. **VACUUM** removes these old files, freeing up storage space and making the data system more efficient.
   - Important: Delta Lake retains old versions of data files for a set time (default is 7 days) to support time travel. If you try to vacuum immediately, it may throw an error because recent files may still be needed. You can override this by adjusting the retention setting.

### 5. **Partitioning**
   - **Partitioning** divides data into subsets based on a specific column, such as **Country** or **CustomerID**. This allows queries to focus on a specific subset of data, improving performance.
   - Example: Partitioning data by country can help when you want to query all data from **Sweden** without needing to scan data from other countries.

### 6. **Aggregations**
   - **Aggregations** (like SUM or COUNT) are calculations across your data. For instance, you can calculate the total number of orders from a specific country or customer.
   - This notebook demonstrates using SQL to calculate such aggregates and compare data between different time versions (before and after changes).

### 7. **SQL Queries**
   - You interact with the Delta Lake data using **SQL queries** to retrieve or manipulate data.
   - Examples in the notebook include querying the total number of orders, retrieving historical data (using `VERSION AS OF`), and running optimization commands.

### 8. **Cluster Configuration**
   - Spark is used to process the data in parallel using multiple nodes (computers). In the notebook, the number of partitions is reduced to better match the available resources of the cluster, improving efficiency.

### Summary:
This notebook helps you manage and optimize large datasets stored in Delta Lake. It allows you to:
- **View historical data versions** (Time Travel)
- **Optimize performance** by reducing the number of small files and sorting data efficiently (OPTIMIZE and ZORDER)
- **Clean up unnecessary files** after updates (VACUUM)
- **Query data** efficiently using SQL, leveraging features like partitioning to speed up queries.

Let's break down **OPTIMIZE** and **VACUUM** using simple examples with tables.

### 1. **OPTIMIZE**
**OPTIMIZE** is used to compact many small files in a Delta table into fewer, larger files. This makes queries run faster because it reduces the number of files Spark has to read.

#### Example Scenario:
Imagine you have a Delta table called `customer_orders` that stores millions of customer orders. Each update or write operation might create many small files. Over time, as more data is added, the number of small files grows, and queries start to slow down because the system has to read a lot of files to retrieve the data.

**Without Optimization**:
```plaintext
Table: customer_orders

Files:
order_part_001.parquet
order_part_002.parquet
order_part_003.parquet
...
order_part_5000.parquet
```
You might have thousands of small files, and every time you query the table (e.g., to get the total orders for a customer), Spark has to scan all those files.

Now, let’s **optimize** this table to improve query performance.

**Optimization Command**:
```sql
OPTIMIZE customer_orders
```

What happens:
- The system will **compact** these files. It will merge smaller files into larger ones, reducing the overall number of files.
  
**After Optimization**:
```plaintext
Table: customer_orders

Files:
order_part_optimized_001.parquet
order_part_optimized_002.parquet
```
Now, instead of thousands of small files, we have just a few large ones, making the queries faster.

#### ZORDER for Even Faster Queries:
You can further optimize the table based on frequently queried columns. For instance, if you often query `customer_orders` by **CustomerID**, you can use **ZORDER** to reorder the data.

**Command with ZORDER**:
```sql
OPTIMIZE customer_orders ZORDER BY (CustomerID)
```
This ensures that rows with the same `CustomerID` are stored together in fewer files, which improves the speed of queries related to `CustomerID`.

---

### 2. **VACUUM**
**VACUUM** cleans up old, unnecessary files that are no longer in use after updates or optimizations. Delta Lake keeps old files to support features like time travel, but over time, these files take up space and are not needed anymore.

#### Example Scenario:
After you update or optimize the `customer_orders` table, there might be old versions of files lying around, which are no longer part of the current table. These old files consume storage space.

Before running **VACUUM**, your table might look like this:

**Files After Updates (Before VACUUM)**:
```plaintext
Files:
order_part_001.parquet    <-- Old file (no longer needed)
order_part_002.parquet    <-- Old file (no longer needed)
order_part_optimized_001.parquet  <-- New file (current data)
order_part_optimized_002.parquet  <-- New file (current data)
```

The old files `order_part_001.parquet` and `order_part_002.parquet` are no longer used in the current table after optimization, but they still exist for time travel or rollback purposes.

To clean up these files, you can run the **VACUUM** command.

**Vacuum Command**:
```sql
VACUUM customer_orders RETAIN 168 HOURS
```

This command tells Delta Lake to delete files that are older than **168 hours** (7 days). You can adjust the number of hours if you want to keep the old files for a shorter or longer period. **After VACUUM**, only the necessary, optimized files remain:

**Files After VACUUM**:
```plaintext
Files:
order_part_optimized_001.parquet  <-- New file (current data)
order_part_optimized_002.parquet  <-- New file (current data)
```

In this case, the old files are removed, freeing up storage space.

---

### Summary with a Simple Analogy:
- **OPTIMIZE** is like cleaning up your workspace by putting all scattered papers (small files) into a few neatly organized folders (larger, compacted files).
- **VACUUM** is like throwing away old drafts or documents you no longer need after reorganizing your workspace.

Together, these operations keep your Delta Lake tables efficient and clean, speeding up query performance and saving storage space.